In [5]:
import sys
sys.path.append('/home/ubuntu/loan_analysis_app/scripts')

import streamlit as st
from analyser import analyse
import os

In [6]:
loan_app_name = "loan_application_form.docx"
cr_name = "commercial_registration.png"
il_name = "industrial_license.png"

# data = analyse(loan_app_name, cr_name, il_name).load_rdata

In [ ]:
from document_ext import Data_extractor

Data_extractor().get(loan_app_name, cr_name, il_name)

TypeError: Data_extractor.get() missing 3 required positional arguments: 'loan_app_name', 'cr_name', and 'il_name'